In [84]:
import pandas as pd
import geopandas as gpd
from geopandas import GeoDataFrame
from shapely.geometry import Point

In [85]:

parliament = pd.read_csv('parlamentarni_2016.csv')
opstina = gpd.read_file('Serbia_admin2.geojson')
district = gpd.read_file('Serbia_admin1.geojson')

In [73]:
#filter out opstinas that are outside of Serbia (foreign ballots)

keep_words = ['Барајево', 'Вождовац',
       'Врачар', 'Гроцка', 'Звездара', 'Земун', 'Лазаревац', 'Младеновац',
       'Нови Београд', 'Обреновац', 'Палилула', 'Раковица',
       'Савски Венац', 'Сопот', 'Стари Град', 'Сурчин', 'Чукарица',
       'Севернобачки округ', 'Бачка Топола', 'Мали Иђош', 'Суботица',
        'Житиште', 'Зрењанин', 'Нова Црња', 'Нови Бечеј', 'Сечањ',
       'Ада', 'Кањижа', 'Кикинда', 'Нови Кнежевац', 'Сента',
       'Чока', 'Алибунар', 'Бела Црква', 'Вршац', 'Ковачица', 'Ковин',
       'Опово', 'Панчево', 'Пландиште', 'Апатин', 'Кула', 'Оџаци',
       'Сомбор', 'Бач', 'Бачка Паланка', 'Бачки Петровац', 'Беочин',
       'Бечеј', 'Врбас', 'Жабаљ', 'Нови Сад', 'Србобран',
       'Сремски Карловци', 'Темерин', 'Тител', 'Инђија', 'Ириг',
       'Пећинци', 'Рума', 'Сремска Митровица', 'Стара Пазова', 'Шид',
       'Богатић', 'Владимирци', 'Коцељева', 'Крупањ', 'Лозница',
       'Љубовија', 'Мали Зворник', 'Шабац', 'Ваљево', 'Лајковац', 'Љиг',
       'Мионица', 'Осечина', 'Уб', 'Велика Плана', 'Смедерево',
       'Смедеревска Паланка', 'Велико Градиште', 'Голубац', 'Жабари',
       'Жагубица', 'Костолац', 'Кучево', 'Мало Црниће', 'Петровац',
       'Пожаревац', 'Аранђеловац', 'Баточина', 'Кнић', 'Крагујевац',
       'Лапово', 'Рача', 'Топола', 'Деспотовац', 'Јагодина', 'Параћин',
       'Рековац', 'Свилајнац', 'Ћуприја', 'Бор', 'Кладово', 'Мајданпек',
       'Неготин', 'Бољевац', 'Зајечар', 'Књажевац', 'Сокобања', 'Ариље',
       'Бајина Башта', 'Косјерић', 'Нова Варош', 'Пожега', 'Прибој',
       'Пријепоље', 'Сјеница', 'Ужице', 'Чајетина', 'Горњи Милановац',
       'Ивањица', 'Лучани', 'Чачак', 'Врњачка Бања', 'Краљево',
       'Нови Пазар', 'Рашка', 'Тутин', 'Александровац', 'Брус',
       'Варварин', 'Крушевац', 'Трстеник', 'Ћићевац', 'Алексинац',
       'Гаџин Хан', 'Дољевац', 'Мерошина', 'Ниш', 'Ражањ', 'Сврљиг',
       'Блаце', 'Житорађа', 'Куршумлија', 'Прокупље', 'Бабушница',
       'Бела Паланка', 'Димитровград', 'Пирот', 'Бојник', 'Власотинце',
       'Лебане', 'Лесковац', 'Медвеђа', 'Црна Трава', 'Босилеград',
       'Бујановац', 'Владичин Хан', 'Врање', 'Прешево', 'Сурдулица',
       'Трговиште', 'Косово Поље', 'Липљан', 'Приштина - град', 'Штрпце',
       'Пећ', 'Гора', 'Ораховац', 'Вучитрн', 'Звечан', 'Зубин Поток',
       'Косовска Митровица', 'Лепосавић', 'Витина', 'Гњилане',
       'Косовска Каменица', 'Ново Брдо']

In [74]:
parliament['Назив општине '] = parliament['Назив општине '].str.strip()
parliament = parliament[parliament['Назив општине '].isin(keep_words)]
parliament.dropna(axis=0, subset=['Адреса бирачког места'], inplace=True)
parliament.shape

(8316, 31)

In [75]:
#drop unnecessary columsn before summing results by opstina, use only party counts

cols_to_drop = ['Рбр. бирачког места ', 'Адреса бирачког места',
       'Coordinates', 'Уписани бирачи ', 'Примqени ', 'Неупотреб-qени ',
       'Гласали ', 'У гласачкој кутији', 'Неважећи ', 'Важећи']

In [76]:
parliament.drop(cols_to_drop, axis = 1, inplace=True)

In [77]:
#group results by opstina
grouped = parliament.groupby('Назив општине ')[parliament.columns[1:]].agg('sum').reset_index()

In [90]:
sp_join = opstina.merge(grouped, left_on='NL_NAME_2', right_on='Назив општине ', indicator=False, how='left')

In [91]:
sp_join.to_file('results_by_opstina.geojson', driver='GeoJSON')

/home/alexander/.cache/pypoetry/virtualenvs/serbia-6Rf6d_AT-py3.9/lib/python3.9/site-packages/geopandas/io/file.py:362: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  pd.Int64Index,


In [7]:
#point in poly using coordinates

parliament[['lat', 'lon']] = parliament['Coordinates'].str.split(',', 1, expand=True)
parliament['lon'] = parliament['lon'].astype(float)
parliament['lat'] = parliament['lat'].astype(float)

In [8]:
geometry = [Point(xy) for xy in zip(parliament.lon, parliament.lat)]
parliament = parliament.drop(['lon', 'lat'], axis=1)
parliament = GeoDataFrame(parliament, crs="EPSG:4326", geometry=geometry)

In [11]:
parliament.to_file("parliament_points.geojson", driver='GeoJSON')

/home/alexander/.cache/pypoetry/virtualenvs/serbia-6Rf6d_AT-py3.9/lib/python3.9/site-packages/geopandas/io/file.py:362: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  pd.Int64Index,


In [9]:
pointInPoly = gpd.sjoin(parliament, opstina[['NAME_2', , op='within') 

/home/alexander/.cache/pypoetry/virtualenvs/serbia-6Rf6d_AT-py3.9/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3309: FutureWarning: The `op` parameter is deprecated and will be removed in a future release. Please use the `predicate` parameter instead.
  if await self.run_code(code, result, async_=asy):
